In [1]:
import os
PROJECT = "ml-practice-260405" 
BUCKET = "ml-practice-260405"
REGION = "us-central1"
MODEL_TYPE = "cnn"
TFVERSION = "1.13" # Tensorflow version

# do not change these
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["MODEL_TYPE"] = MODEL_TYPE
os.environ["TFVERSION"] = TFVERSION  

In [2]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Input functions to read JPEG images

The key difference between this notebook and [the MNIST one](./mnist_models.ipynb) is in the input function.
In the input function here, we are doing the following:
* Reading JPEG images, rather than 2D integer arrays.
* Reading in batches of batch_size images rather than slicing our in-memory structure to be batch_size images.
* Resizing the images to the expected HEIGHT, WIDTH. Because this is a real-world dataset, the images are of different sizes. We need to preprocess the data to, at the very least, resize them to constant size.

## Run as a Python module

Let's first run it locally for a short while to test the code works. Note the --model parameter

In [3]:
%%bash
rm -rf flowersmodel.tar.gz flowers_trained
gcloud ml-engine local train \
    --module-name=flowersmodel_augment_tf_v_1_3.task \
    --package-path=./flowersmodel_augment_tf_v_1_3 \
    -- \
    --output_dir=./flowers_trained \
    --train_steps=5\
    --learning_rate=0.01 \
    --batch_size=2 \
    --model=$MODEL_TYPE \
    --augment \
    --train_data_path=gs://cloud-ml-data/img/flowers_photos/train_set.csv \
    --eval_data_path=gs://cloud-ml-data/img/flowers_photos/eval_set.csv

ERROR: (gcloud.ml-engine.local.train) unrecognized arguments:
  Understanding
  with
  TensorFlow
  on
  GCP/Week_2/Lab_5_Image
  Augmentation
  in
  TensorFlow/Practice/flowersmodel_augment_tf_v_1.3
  To search the help text of gcloud commands, run:
  gcloud help -- SEARCH_TERMS


CalledProcessError: Command 'b'rm -rf flowersmodel.tar.gz flowers_trained\ngcloud ml-engine local train \\\n    --module-name=flowersmodel_augment_tf_v_1.3.task \\\n    --package-path=${PWD}/flowersmodel_augment_tf_v_1.3 \\\n    -- \\\n    --output_dir=${PWD}/flowers_trained \\\n    --train_steps=5\\\n    --learning_rate=0.01 \\\n    --batch_size=2 \\\n    --model=$MODEL_TYPE \\\n    --augment \\\n    --train_data_path=gs://cloud-ml-data/img/flowers_photos/train_set.csv \\\n    --eval_data_path=gs://cloud-ml-data/img/flowers_photos/eval_set.csv\n'' returned non-zero exit status 2.

Now, let's do it on ML Engine. Note the --model parameter

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/flowers/trained_${MODEL_TYPE}
JOBNAME=flowers_${MODEL_TYPE}_$(date -u + %y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
    --region=$REGION \
    --module-name=flowersmodel_augment_tf_v_1_3.task \
    --package-path=./flowersmodel_augment_tf_v_1_3 \
    --job-dir=$OUTDIR \
    --staging-bucket=gs://$BUCKET \
    --scale-tier=BASIC_GPU \
    --runtime-version=$TFVERSION \
    -- \
    --output_dir=$OUTDIR \
    --train_steps=1000 \
    --learning_rate=0.01 \
    --batch_size=40 \
    --model=$MODEL_TYPE \
    --augment \
    --batch_norm \
    --train_data_path=gs://cloud-ml-data/img/flower_photos/train_set.csv \
    --eval_data_path=gs://cloud-ml-data/img/flower_photos/eval_set.csv

## Deploying and predicting with model

Deploy the model:

In [ ]:
%%bash
MODEL_NAME="flowers"
MODEL_VERSION=${MODEL_TYPE}
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/flowers/trained_${MODEL_TYPE}/export/exporter | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ...this will take a few minutes"
#gcloud ml-engine versions delete --quiet ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions ${REGION}
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=$TFVERSION

To predict with the model, let's take one of the example images that is available on Google Cloud Storage <img src="http://storage.googleapis.com/cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg" />

The online prediction service expects images to be base64 encoded as described [here](https://cloud.google.com/ml-engine/docs/tensorflow/online-predict#binary_data_in_prediction_input).

In [ ]:
%%bash
IMAGE_URL=gs://cloud-ml-data/img/flower_photos/sunflowers/1022552002_2b93faf9e7_n.jpg

# Copy the image to local disk.
gsutil cp $IMAGE_URL flower.jpg

# Base64 encode and create request message in json format.
python -c 'import base64, sys, json; img = base64.b64encode(open("flower.jpg", "rb").read()).decode(); print(json.dumps({"image_bytes":{"b64": img}}))' &> request.json

Send it to the prediction service

In [ ]:
%%bash
gcloud ml-engine predict \
    --model=flowers \
    --version=${MODEL_TYPE} \
    --json-instances=./request.json